In [1]:
from pydantic_ai import Agent
from pydantic import BaseModel, Field
import os
from typing import List, Optional
from enum import Enum

In [2]:
# Required for PydanticAI to work with Jupyter (nested event loops)
import nest_asyncio

nest_asyncio.apply()

In [3]:
# Define enums for standardized fields
class Priority(str, Enum):
    HIGH = "High"
    MEDIUM = "Medium"
    LOW = "Low"


class StoryStatus(str, Enum):
    TODO = "To Do"
    IN_PROGRESS = "In Progress"
    IN_REVIEW = "In Review"
    DONE = "Done"


class StorySize(str, Enum):
    XS = "XS (1 point)"
    S = "S (2 points)"
    M = "M (3 points)"
    L = "L (5 points)"
    XL = "XL (8 points)"
    XXL = "XXL (13 points)"

In [4]:
# Define acceptance criteria as a separate model
class AcceptanceCriterion(BaseModel):
    description: str = Field(description="The specific criterion that must be met")
    test_case: str = Field(description="How this criterion can be verified or tested")


# Define the complete user story model
class UserStory(BaseModel):
    title: str = Field(description="Short title of the user story")
    description: str = Field(
        description="The standard user story format: 'As a [type of user], I want [goal] so that [benefit]'"
    )
    acceptance_criteria: List[AcceptanceCriterion] = Field(
        description="List of specific criteria that must be met for the story to be considered complete"
    )
    priority: Priority = Field(description="The business priority of this story")
    size: StorySize = Field(description="Story point estimation")
    owner: Optional[str] = Field(description="Team member responsible for implementing the story")
    assignee: Optional[str] = Field(description="Team member currently assigned to the story")
    status: StoryStatus = Field(description="Current status in the workflow")
    epic: Optional[str] = Field(description="The larger initiative or epic this story belongs to")
    sprint: Optional[str] = Field(description="The sprint this story is assigned to")
    dependencies: Optional[List[str]] = Field(
        description="IDs or names of other stories this one depends on"
    )
    tags: Optional[List[str]] = Field(description="Relevant tags or labels for categorization")
    notes: Optional[str] = Field(
        description="Additional contextual information or implementation notes"
    )

In [ ]:
# Create a Bedrock agent using Claude
bedrock_agent = Agent(
    model="bedrock:anthropic.claude-3-sonnet-20240229-v1:0",
    model_settings={"temperature": 0.7, "max_tokens": 2000},
    system_prompt="""You are an experienced Agile Product Owner with expertise in writing
    high-quality user stories. When asked for an example, provide a realistic,
    detailed user story.""",
    result_type=UserStory,
)


# Test the agent and return the raw Pydantic model
def get_user_story_example(prompt: str):
    agent = bedrock_agent
    response = agent.run_sync(prompt)
    return response.data

In [8]:
# Get the user story
prompt = "Generate a complete example user story for a financial app that helps users track and categorize their expenses."
user_story = get_user_story_example(prompt)

print(f"\nRetrieved user story:\n{user_story.model_dump_json(indent=4)}")


Retrieved user story:
{
    "title": "Expense Categorization",
    "description": "As a user of the financial app, I want to be able to categorize my expenses by type (e.g. food, transportation, entertainment) so that I can better understand where my money is going.",
    "acceptance_criteria": [
        {
            "description": "User can create custom categories for expenses",
            "test_case": "Create several test categories and verify they are saved correctly"
        },
        {
            "description": "User can assign an expense to one or more categories",
            "test_case": "Add an expense and assign it to multiple categories, verify it shows up under each category"
        },
        {
            "description": "User can view expenses grouped by category",
            "test_case": "Filter the expense list to only show categories, verify the totals per category are calculated correctly"
        }
    ],
    "priority": "High",
    "size": "M (3 points)",
  

In [9]:
# Examine the Pydantic model
user_story

UserStory(title='Expense Categorization', description='As a user of the financial app, I want to be able to categorize my expenses by type (e.g. food, transportation, entertainment) so that I can better understand where my money is going.', acceptance_criteria=[AcceptanceCriterion(description='User can create custom categories for expenses', test_case='Create several test categories and verify they are saved correctly'), AcceptanceCriterion(description='User can assign an expense to one or more categories', test_case='Add an expense and assign it to multiple categories, verify it shows up under each category'), AcceptanceCriterion(description='User can view expenses grouped by category', test_case='Filter the expense list to only show categories, verify the totals per category are calculated correctly')], priority=<Priority.HIGH: 'High'>, size=<StorySize.M: 'M (3 points)'>, owner='<UNKNOWN>', assignee='<UNKNOWN>', status=<StoryStatus.TODO: 'To Do'>, epic='Personal Finance Tracking', sp

In [10]:
vars(user_story)

{'title': 'Expense Categorization',
 'description': 'As a user of the financial app, I want to be able to categorize my expenses by type (e.g. food, transportation, entertainment) so that I can better understand where my money is going.',
 'acceptance_criteria': [AcceptanceCriterion(description='User can create custom categories for expenses', test_case='Create several test categories and verify they are saved correctly'),
  AcceptanceCriterion(description='User can assign an expense to one or more categories', test_case='Add an expense and assign it to multiple categories, verify it shows up under each category'),
  AcceptanceCriterion(description='User can view expenses grouped by category', test_case='Filter the expense list to only show categories, verify the totals per category are calculated correctly')],
 'priority': <Priority.HIGH: 'High'>,
 'size': <StorySize.M: 'M (3 points)'>,
 'owner': '<UNKNOWN>',
 'assignee': '<UNKNOWN>',
 'status': <StoryStatus.TODO: 'To Do'>,
 'epic': '

In [15]:
user_story.acceptance_criteria[2].test_case

'Filter the expense list to only show categories, verify the totals per category are calculated correctly'